In [1]:
!ls Datos_split/

koi_labels_candidate.csv  koi_light_curves_candidate.csv
koi_labels_test.csv	  koi_light_curves_test.csv
koi_labels_train.csv	  koi_light_curves_train.csv
koi_labels_val.csv	  koi_light_curves_val.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import FATS
import numpy as np
import time

time_ex = np.arange(3000)
magnitude_ex = np.random.rand(3000)

lc_example = np.array([magnitude_ex, time_ex])

time_start = time.time()
#a = FATS.FeatureSpace(Data=['magnitude','time'],excludeList= ['SlottedA_length','PeriodLS','StetsonK_AC',"Period_fit","Psi_CS","Psi_eta"])
#SACAR DE EXCLUDE PERIODLS
a = a.calculateFeature(lc_example)
print("Termino en %f segundos"%(time.time()-time_start))
a.result(method='dict')

Termino en 12.516811 segundos


{'Amplitude': 0.47528001055681457,
 'AndersonDarling': 1.0,
 'Autocor_length': 1.0,
 'Con': 0.0,
 'Eta_e': 1.9737367290234138,
 'FluxPercentileRatioMid20': 0.22316396701247973,
 'FluxPercentileRatioMid35': 0.37293943827141829,
 'FluxPercentileRatioMid50': 0.53216329552147645,
 'FluxPercentileRatioMid65': 0.7092926584782373,
 'FluxPercentileRatioMid80': 0.88565351992636576,
 'Freq1_harmonics_amplitude_0': 0.028910907649750524,
 'Freq1_harmonics_amplitude_1': 0.0081067897076466874,
 'Freq1_harmonics_amplitude_2': 0.0072152260786718632,
 'Freq1_harmonics_amplitude_3': 0.0048802292586874046,
 'Freq1_harmonics_rel_phase_0': 0.0,
 'Freq1_harmonics_rel_phase_1': -0.47610252076985216,
 'Freq1_harmonics_rel_phase_2': -1.20288479104943,
 'Freq1_harmonics_rel_phase_3': 1.0603130320323844,
 'Freq2_harmonics_amplitude_0': 0.028601910690315983,
 'Freq2_harmonics_amplitude_1': 0.0083295212319006544,
 'Freq2_harmonics_amplitude_2': 0.0060294940526573644,
 'Freq2_harmonics_amplitude_3': 0.0019914504075

In [4]:
features = a.result(method='dict').keys()
print("Caracteristicas: ",len(features))

('Caracteristicas: ', 48)


In [1]:
import pandas as pd
df_metadata = pd.read_csv('OwnFats/columnas_metadatos.csv')
df_metadata_stellar = pd.read_csv('OwnFats/columnas_metadatos_estrella.csv')

df_metadata_stellar.head()

,KOI Name,Period,Transit Depth,Teq,KOI count,Planet Radius,Teff,log(g),Metallicity,Stellar Radius,Stellar Mass
0,K00889.01,8.884923,16053.4,770.0,1,10.51,5332.0,4.500,-0.08,0.843,0.821
1,K01009.01,5.092465,254.0,844.0,1,1.17,5037.0,4.547,-0.20,0.751,0.727
2,K07621.01,275.073650,228.9,303.0,1,1.86,5604.0,4.226,-0.12,1.187,0.864
3,K06252.01,8.460439,76412.9,804.0,1,36.62,5951.0,4.638,-1.06,0.702,0.796
4,K00958.01,21.761298,1300.7,486.0,1,2.53,4859.0,4.612,0.00,0.720,0.781


In [2]:
df_metadata.head()

,KOI Name,Period,Transit Depth,Teq,KOI count,Planet Radius
0,K00889.01,8.884923,16053.4,770.0,1,10.51
1,K01009.01,5.092465,254.0,844.0,1,1.17
2,K07621.01,275.073650,228.9,303.0,1,1.86
3,K06252.01,8.460439,76412.9,804.0,1,36.62
4,K00958.01,21.761298,1300.7,486.0,1,2.53


In [5]:
import numpy as np
np.sum((pd.isnull(df_metadata["Transit Depth"]))*1)

59

In [7]:
np.sum((pd.isnull(df_metadata_stellar["Metallicity"]))*1)

65

In [14]:
a = df[(df["KOI Name"] == "K00889.01")].values[0][1:]
a

array([8.884922995, 16053.4, 770.0, 1], dtype=object)

In [16]:
import numpy as np
np.hstack((a,a))

array([8.884922995, 16053.4, 770.0, 1, 8.884922995, 16053.4, 770.0, 1], dtype=object)

## OWN FATS

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import LinearRegression

def amplitude(magnitudes):
    return 0.5 * (np.max(magnitudes) - np.min(magnitudes))

def median_absolute_deviation(magnitudes):
    median = np.median(magnitudes)
    deviations = magnitudes - median
    absolute_deviations = np.absolute(deviations)

    return np.median(absolute_deviations)

def residual_bright_faint_ratio(magnitudes):    # median as a fit
    mean = np.mean(magnitudes)

    brighter = magnitudes[magnitudes > mean]
    fainter = magnitudes[magnitudes < mean]

    resid_brighter = np.mean(np.square(brighter - mean))
    resid_fainter = np.mean(np.square(fainter - mean))

    ratio = resid_fainter / resid_brighter

    return ratio

def own_fats(sequence):
    time_ex =  np.arange(len(sequence))

    minim=np.min(sequence)
    maxim = np.max(sequence)
    mean = np.mean(sequence)
    std = np.std(sequence)
    iqr = stats.iqr(sequence) #q31
    skew = stats.skew(sequence)
    kurt = stats.kurtosis(sequence)
    q1 = np.percentile(sequence, 25)
    q2 = np.percentile(sequence, 50)
    model = LinearRegression(normalize=True,n_jobs=-1)
    model.fit(time_ex.reshape(-1,1),sequence)
    slope = model.coef_[0]
    #new features
    ampl = amplitude(sequence)
    mad = median_absolute_deviation(sequence)
    br_fa = residual_bright_faint_ratio(sequence)
    median = np.median(sequence)
    return np.array([minim,maxim,mean,std,iqr,skew,kurt,q1,q2,slope,ampl,mad,br_fa,median])

def metadata_columns(match,array,stellar=False):
    if stellar:
        df = pd.read_csv('OwnFats/columnas_metadatos_estrella.csv')
    else:
        df = pd.read_csv('OwnFats/columnas_metadatos.csv')
    metadata = df[(df["KOI Name"] == match)].values[0][1:]
    return np.hstack((array,metadata))

In [2]:
import pandas as pd
df_train = pd.read_csv("Linea_Tendencia/koi_light_curves_full_train.csv",header=None) #koi_light_curves
#df_train = pd.read_csv("Resample/koi_light_curves_resample2_train.csv",header=None) #koi_light_curves sample
df_train_label = pd.read_csv("Datos_split/koi_labels_train.csv") #koi_light_curves


In [3]:
import numpy as np
from scipy.signal import resample

X_fats1 = []
X_fats2 = []

X = df_train.values
for sequence,match in zip(X,df_train_label["KOI Name"]):
    aux = own_fats(sequence)
    final1 = metadata_columns(match,aux,True)
    final2 = metadata_columns(match,aux,False)

    X_fats1.append(final1)
    X_fats2.append(final2)
print("YA SE ExtraYO TODO! ahora escribir..")

/etc/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/etc/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


YA SE ExtraYO TODO! ahora escribir..


In [4]:
import csv
b = open('koi_light_curves_FATS2_metedatastellar_train.csv', 'w') #cambiar "w" por "a"
a = csv.writer(b)
a.writerows(X_fats1)
b.close()

b = open('koi_light_curves_FATS2_metadata_train.csv', 'w') #cambiar "w" por "a"
a = csv.writer(b)
a.writerows(X_fats2)
b.close()

In [9]:
df = pd.read_csv('koi_light_curves_FATS2_metadata_train.csv',header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,-41.321,16.889,-0.007361,2.254558,1.469008,-6.180130,64.167718,-0.640008,0.082,1.633369e-06,29.105,0.735383,4.038914,0.082,8.884923,16053.4,770.0,1,10.51
1,-4.632,12.196,0.088144,0.941973,1.170000,0.171710,2.179614,-0.508000,0.150,-4.815129e-06,8.414,0.573000,1.151498,0.150,5.092465,254.0,844.0,1,1.17
2,-139.123,21.879,-0.714502,5.838941,1.411000,-13.070066,233.998145,-0.761000,-0.036,-2.554466e-05,80.501,0.704000,45.176895,-0.036,275.073650,228.9,303.0,1,1.86
3,-21.000,164.128,-0.010487,1.287897,1.288000,34.431573,4060.501884,-0.651000,-0.005,1.984248e-07,92.564,0.644000,0.541691,-0.005,0.539366,145.0,5016.0,1,9.19
4,-5.785,7.993,0.025688,0.996821,1.260000,0.217364,1.039805,-0.626000,0.023,1.001995e-06,6.889,0.630000,0.919011,0.023,36.267167,627.1,347.0,1,1.50


In [30]:
df_val_label[""]

,NExScI Disposition,KOI Name,Kepler Name,Planet Radius,Stellar Radius,Metallicity,Stellar Mass
0,FALSE POSITIVE,K07623.01,NaN,34.14,20.848,0.07,3.584
1,FALSE POSITIVE,K00362.01,NaN,14.93,1.050,-0.14,1.012
2,CONFIRMED,K01007.01,Kepler-748 b,2.01,0.961,-0.08,1.002
3,FALSE POSITIVE,K04927.01,NaN,37.88,1.769,0.14,1.338
4,CONFIRMED,K02634.01,Kepler-1299 b,2.42,0.763,-0.40,0.853


In [5]:
#Val
import time

#df_val = pd.read_csv("Datos_split/koi_light_curves_val.csv",header=None)
df_val = pd.read_csv("Linea_Tendencia/koi_light_curves_full_val.csv",header=None) #koi_light_curves

df_val_label = pd.read_csv("Datos_split/koi_labels_val.csv") #koi_light_curves

X_fats1 = []
X_fats2 = []

X = df_val.values
final_time = []

for sequence,match in zip(X,df_val_label["KOI Name"]):
    aux = own_fats(sequence)
    final1 = metadata_columns(match,aux,True)
    final2 = metadata_columns(match,aux,False)

    X_fats1.append(final1)
    X_fats2.append(final2)

print("Termino cada secuencia en %f aprox segundos"%(np.mean(final_time)))

/etc/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/etc/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Termino cada secuencia en nan aprox segundos


In [6]:
import csv
b = open('koi_light_curves_FATS2_metedatastellar_val.csv', 'w') #cambiar "w" por "a"
a = csv.writer(b)
a.writerows(X_fats1)
b.close()

b = open('koi_light_curves_FATS2_metadata_val.csv', 'w') #cambiar "w" por "a"
a = csv.writer(b)
a.writerows(X_fats2)
b.close()

In [ ]:
#test

In [7]:
#df_test = pd.read_csv("Datos_split/koi_light_curves_test.csv",header=None)
df_test = pd.read_csv("Linea_Tendencia/koi_light_curves_full_test.csv",header=None) #koi_light_curves

df_test_label = pd.read_csv("Datos_split/koi_labels_test.csv") #koi_light_curves

X_fats1 = []
X_fats2 = []

X = df_test.values
final_time = []

for sequence,match in zip(X,df_test_label["KOI Name"]):
    aux = own_fats(sequence)
    final1 = metadata_columns(match,aux,True)
    final2 = metadata_columns(match,aux,False)

    X_fats1.append(final1)
    X_fats2.append(final2)

print("Termino cada secuencia en %f aprox segundos"%(np.mean(final_time)))

/etc/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/etc/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Termino cada secuencia en nan aprox segundos


In [8]:
import csv
b = open('koi_light_curves_FATS2_metedatastellar_test.csv', 'w') #cambiar "w" por "a"
a = csv.writer(b)
a.writerows(X_fats1)
b.close()

b = open('koi_light_curves_FATS2_metadata_test.csv', 'w') #cambiar "w" por "a"
a = csv.writer(b)
a.writerows(X_fats2)
b.close()